In [ ]:
%%time
import json
import os
import warnings

import pandas as pd
import pulumi
from datarobot_predict.deployment import predict

import datarobot as dr
import pulumi_datarobot as datarobot
from pulumi import automation as auto

if not os.getenv("DATAROBOT_NOTEBOOK_IMAGE"):
    print("not running in DataRobot Notebook")
    from dotenv import load_dotenv
    load_dotenv("../.env", override=True)
else:
    print("running in DataRobot Notebook")
    os.environ["PULUMI_CONFIG_PASSPHRASE"] = "dr"
warnings.filterwarnings("ignore")

client = dr.Client()

pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)
pd.set_option("display.width", 1000)
pd.set_option("display.max_colwidth", 1000)
pd.set_option("display.precision", 8)

CPU times: user 415 ms, sys: 188 ms, total: 603 ms
Wall time: 1.43 s


In [2]:
model_id = "67bb0f8070b10e0e4071f347"
usecase_id = os.getenv("DATAROBOT_DEFAULT_USE_CASE")
stack_name = "mlops-bleedout"
project_name = "dr-workshop"

In [3]:
def stack_up(project_name: str, stack_name: str, program: callable) -> auto.Stack:
    # create (or select if one already exists) a stack that uses our inline program
    stack = auto.create_or_select_stack(
        stack_name=stack_name, project_name=project_name, program=program
    )

    stack.refresh(on_output=print)

    stack.up(on_output=print)
    return stack


def destroy_project(stack: auto.Stack):
    """Destroy pulumi project"""
    stack_name = stack.name
    stack.destroy(on_output=print)

    stack.workspace.remove_stack(stack_name)
    print(f"stack {stack_name} in project removed")


def make_deployment():
    """Deploy a trained model onto DataRobot."""
    user_name = client.get("account/info").json()["email"].split("@")[0]
    registered_model = datarobot.RegisteredModelFromLeaderboard(
        resource_name=f"[{user_name}]-registered-model",
        model_id=model_id,
        name=f"[{user_name}]-registered-model",
        use_case_ids=[usecase_id],
    )
    registered_model_id = registered_model.id
    registered_model_version_id = registered_model.version_id
    prediction_environment = datarobot.PredictionEnvironment(
        resource_name=f"[{user_name}]-prediction-environment",
        name=f"[{user_name}]-prediction-environment",
        batch_jobs_max_concurrent=20,
        platform="datarobotServerless",
        supported_model_formats=[
            "datarobot",
            # "customModel",
        ],
    )
    prediction_environment_id = prediction_environment.id
    deployment = datarobot.Deployment(
        resource_name=f"[{user_name}]-deployment",
        label=f"[{user_name}]-deployment",
        registered_model_version_id=registered_model_version_id,
        prediction_environment_id=prediction_environment_id,
        drift_tracking_settings={
            "feature_drift_enabled": True,
            "target_drift_enabled": True,
        },
        association_id_settings={
            "auto_generate_id": False,
            "column_names": ["association_id"],
            "required_in_prediction_requests": True,
        },
        health_settings={
            "data_drift": {
                "time_interval": "P180D",
            },
        },
        challenger_models_settings={
            "enabled": True,
        },
        predictions_by_forecast_date_settings={
            "enabled": True,
            "column_name": "date",
            "datetime_format": "%Y-%m-%d",  # 2022-01-01
        },
        predictions_data_collection_settings={
            "enabled": True,
        },
        batch_monitoring_settings={
            "enabled": False,
        },
        segment_analysis_settings={
            "enabled": True,
            "attributes": [],
        },
        use_case_ids=[usecase_id],
    )
    pulumi.export("registered_model_id", registered_model_id)
    pulumi.export("registered_model_version_id", registered_model_version_id)
    pulumi.export("prediction_environment_id", prediction_environment_id)
    pulumi.export("deployment_id", deployment.id)

In [4]:
stack = stack_up(project_name, stack_name, program=make_deployment)

Refreshing (mlops-bleedout)

View Live: https://app.pulumi.com/ironerumi/dr-workshop/mlops-bleedout/updates/1


Resources:

Duration: 1s



I0000 00:00:1740323919.701718 35719963 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


Updating (mlops-bleedout)

View Live: https://app.pulumi.com/ironerumi/dr-workshop/mlops-bleedout/updates/2


 +  pulumi:pulumi:Stack dr-workshop-mlops-bleedout creating (0s)
@ Updating....
 +  datarobot:index:RegisteredModelFromLeaderboard [yifu.gu+dic.corporation]-registered-model creating (0s)
 +  datarobot:index:PredictionEnvironment [yifu.gu+dic.corporation]-prediction-environment creating (0s)
@ Updating....
 +  datarobot:index:PredictionEnvironment [yifu.gu+dic.corporation]-prediction-environment created (1s)
@ Updating.............................................................................
 +  datarobot:index:RegisteredModelFromLeaderboard [yifu.gu+dic.corporation]-registered-model created (74s)
 +  datarobot:index:Deployment [yifu.gu+dic.corporation]-deployment creating (0s)
@ Updating...........
 +  datarobot:index:Deployment [yifu.gu+dic.corporation]-deployment created (8s)
@ Updating.........
 +  pulumi:pulumi:Stack dr-workshop-mlops-bleedout created (84s)
Outputs:
   

I0000 00:00:1740324011.190733 35719963 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1740324012.393686 35719963 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers
I0000 00:00:1740324013.410563 35719963 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


In [5]:
result = stack.outputs()
deployment_id = result["deployment_id"].value
deployment = dr.Deployment.get(deployment_id)

### 予測

In [6]:
df_test = pd.read_csv("../data/mlops_bleed_out_test.csv")
display(df_test.head())
df_result, _ = predict(deployment, df_test)
display(df_result.head())

,種別,号機,コーティング材成分A,コーター部温度,コーター部相対湿度,ポンプ圧力,乾燥ゾーン1温度,乾燥ゾーン2温度,UV照度,ランプ点灯時間,チャンバー内O2濃度,UVロール温度,テンション_1,テンション_2,テンション_巻き取り,epc_diff_1,epc_diff_2,association_id,date
0,製造,YC-08,99.19711436,28.82,51.3,0.9,120.26,122.52,1021.4,2079,0.01165,89.08,3.128,2.906,3.193,-0.14204040,0.02542395,2024-01-01-00296,2024-01-01
1,製造,YC-08,99.72265556,27.89,50.4,0.9,120.01,122.01,1020.0,674,0.01094,89.02,2.559,2.496,3.103,-0.06088695,0.02949297,2024-01-01-00303,2024-01-01
2,製造,YC-08,100.32124994,27.38,50.2,0.9,120.00,122.05,1020.2,288,0.01046,89.04,3.563,2.054,3.116,-0.03900678,0.07430120,2024-01-01-00306,2024-01-01
3,製造,YC-08,99.19587922,27.69,50.6,0.9,120.14,122.02,1020.2,968,0.01144,89.04,3.219,2.876,3.116,-0.18418339,0.05472725,2024-01-01-00316,2024-01-01
4,製造,YC-08,99.90226845,28.36,50.6,0.9,120.09,122.01,1020.1,1038,0.01103,89.10,4.142,3.148,3.116,-0.07379887,0.09959071,2024-01-01-00320,2024-01-01


,品質不良_True_PREDICTION,品質不良_False_PREDICTION,品質不良_PREDICTION,THRESHOLD,POSITIVE_CLASS,DEPLOYMENT_APPROVAL_STATUS,association_id
0,0.08646156,0.91353844,False,0.5,True,APPROVED,2024-01-01-00296
1,0.14277684,0.85722316,False,0.5,True,APPROVED,2024-01-01-00303
2,0.05023774,0.94976226,False,0.5,True,APPROVED,2024-01-01-00306
3,0.26079261,0.73920739,False,0.5,True,APPROVED,2024-01-01-00316
4,0.08285966,0.91714034,False,0.5,True,APPROVED,2024-01-01-00320


### 実績値をアップロード

In [7]:
dr.Client()
df_actuals = pd.read_csv("../data/mlops_bleed_out_actuals.csv")
deployment.submit_actuals(df_actuals)
# json_actual = df_actuals.rename(
#     columns={"association_id": "associationId", "actual_value": "actualValue"}
# ).to_dict(orient="records")
# response = client.post(
#     f"deployments/{deployment_id}/actuals/fromJSON/", json={"data": json_actual}
# )
# print(response.status_code)

### デプロイなどリソースの削除

In [ ]:
# destroy_project(stack)

Destroying (mlops-bleedout)

View Live: https://app.pulumi.com/ironerumi/dr-workshop/mlops-bleedout/updates/7


@ Destroying....
 -  datarobot:index:Deployment [yifu.gu+dic.corporation]-deployment deleting (0s)
@ Destroying....
 -  datarobot:index:Deployment [yifu.gu+dic.corporation]-deployment deleted (1s)
@ Destroying....
 -  datarobot:index:PredictionEnvironment [yifu.gu+dic.corporation]-prediction-environment deleting (0s)
 -  datarobot:index:RegisteredModelFromLeaderboard [yifu.gu+dic.corporation]-registered-model deleting (0s)
@ Destroying....
 -  datarobot:index:PredictionEnvironment [yifu.gu+dic.corporation]-prediction-environment deleted (1s)
 -  datarobot:index:RegisteredModelFromLeaderboard [yifu.gu+dic.corporation]-registered-model deleted (1s)
 -  pulumi:pulumi:Stack dr-workshop-mlops-bleedout deleting (0s)
@ Destroying....
 -  pulumi:pulumi:Stack dr-workshop-mlops-bleedout deleted (0.38s)
Outputs:
  - deployment_id              : "67bb22a01533826097b10fb4"
  - prediction_